In [1]:
import zipfile as zp
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression 
import ipyparallel as ipp
import gc

In [2]:
df = pd.read_csv('train.zip', index_col = 0)
df

,visits
id,
1,30 84 126 135 137 179 242 342 426 456 460 462...
2,24 53 75 134 158 192 194 211 213 238 251 305 ...
3,51 143 173 257 446 491 504 510 559 616 719 73...
4,1 20 22 92 124 149 211 335 387 390 406 460 48...
5,34 51 56 106 110 121 163 233 266 275 345 359 ...
...,...
299996,6 12 15 30 33 48 49 50 55 56 70 74 76 83 84 9...
299997,4 7 12 18 19 35 41 53 55 60 75 77 81 82 91 95...
299998,1 9 15 17 29 31 42 49 62 74 85 105 109 113 11...


# Обработка всего сета

In [8]:
id_i_visit = pd.DataFrame({'Days': [(1 if i in days else 0) for i in range(3) ]})
id_i_visit

,Days
0,0
1,0
2,1


visits     30 84 126 135 137 179 242 342 426 456 460 462...
Name: 1, dtype: object

In [155]:
client = ipp.Client()
print(len(client))

8


In [156]:
lbview = client.load_balanced_view()

In [157]:
def PreProcessing(df):
    import pandas as pd
    import gc
    days = list(map(int, (df.str.split(' ')[0])[1:]))
    id_i_visits = [(1 if i in days else 0) for i in range(1, 1099 + 1)]
    gc.collect()
    return id_i_visits

In [ ]:
%%time
gc.collect()
import csv
res = {}
for i in df.index.tolist():
    res_ = lbview.apply(PreProcessing, df.loc[i])
    res[i] = res_
client.wait()

In [ ]:
a = min(df.index.tolist())
b = max(df.index.tolist()) +1

In [ ]:
df_afterProc = pd.DataFrame({'id' : df.index.tolist(), 
                             'Days' : [[0] * 1099] * (b - 1)}).set_index('id')
df_afterProc

In [ ]:
for i in df_afterProc.index.tolsit():
    df_afterProc.loc[1].Days =  res[1].get()

In [ ]:
df_afterProc.to_csv('AfterProcessingData.csv')

In [164]:

df_afterProc.loc[1].Days =  res[1].get()

In [165]:
df_afterProc

,Days
id,
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
299996,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
299998,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [166]:
pd.DataFrame({'data': df_afterProc.loc[1].Days}).value_counts()

data
0       1070
1         29
dtype: int64